# ISOLATION FOREST 

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer

# Step 1: Import the data
data = pd.read_csv(r'S:\CODING NOTES\PYTHON3.0\CSV files\preprocessed Sensor_data.csv')

# Step 2: Split the data into training and testing sets
features = data.drop(['machine_status', 'date'], axis=1)  # Exclude label and date columns
labels = data['machine_status']

# Step 3: Handle missing values
imputer = SimpleImputer(strategy='mean')
X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Step 3: Train the anomaly detection model
model = IsolationForest(contamination=0.05)  # Adjust contamination parameter as needed
model.fit(X_train)

# Step 4: Evaluate the model
train_scores = model.decision_function(X_train)
test_scores = model.decision_function(X_test)

# Step 5: Visualize the results
plt.figure(figsize=(12, 6))
plt.hist(train_scores, bins=50, label='Normal')
plt.hist(test_scores, bins=50, label='Anomaly')
plt.xlabel('Anomaly Score')
plt.ylabel('Frequency')
plt.legend()
plt.show()


NameError: name 'X_train' is not defined

In [5]:
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder


# Step 7: Correct any misspelled labels
y_train[y_train == 'NORMA'] = 'NORMAL'
y_test[y_test == 'NORMA'] = 'NORMAL'

# Step 8: Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")  # Ignore the UndefinedMetricWarning
    train_cm = confusion_matrix(y_train_encoded, train_predictions, labels=[0, 1])
    test_cm = confusion_matrix(y_test_encoded, test_predictions, labels=[0, 1])

train_report = classification_report(y_train_encoded, train_predictions, labels=[0, 1])
test_report = classification_report(y_test_encoded, test_predictions, labels=[0, 1])

print("Train Classification Report:\n", train_report)
print("Test Classification Report:\n", test_report)

# Step 9: Visualize confusion matrix
labels = label_encoder.classes_

plt.figure(figsize=(10, 6))
plt.subplot(1, 2, 1)
sns.heatmap(train_cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.title('Train Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')

plt.subplot(1, 2, 2)
sns.heatmap(test_cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.title('Test Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')

plt.tight_layout()
plt.show()

NameError: name 'y_train' is not defined

# GNN

In [3]:
import dgl
import pandas as pd
import numpy as np
import torch
import torch.nn as nn


# Step 1: Read the preprocessed data
data = pd.read_csv(r'S:\CODING NOTES\PYTHON3.0\CSV files\preprocessed Sensor_data.csv')

# Step 2: Extract sensor readings and machine status
sensor_data = data.iloc[:, :-2].values
machine_status = data['machine_status'].values

# Step 3: Create graph and add nodes
num_nodes = sensor_data.shape[0]
graph = dgl.DGLGraph()
graph.add_nodes(num_nodes)

# Step 4: Add node features (sensor readings)
graph.ndata['features'] = torch.tensor(sensor_data, dtype=torch.float32)

# Step 5: Create edges based on time intervals
# Assuming each row represents a time interval, create edges between consecutive time intervals
src = np.arange(num_nodes - 1)
dst = np.arange(1, num_nodes)
graph.add_edges(src, dst)

# Step 6: Set machine status as node labels
machine_status = np.where(machine_status == 'NORMAL', 0, 1)  # Convert labels to numeric type
graph.ndata['label'] = torch.tensor(machine_status, dtype=torch.long)

# Print the graph information
print(graph)




Graph(num_nodes=220320, num_edges=220319,
      ndata_schemes={'features': Scheme(shape=(51,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={})


C:\Users\SUJAN\anaconda3\envs\myenv\lib\site-packages\dgl\heterograph.py:92: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning(


In [4]:
import dgl
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Step 7: Split the graph into training and testing sets
train_mask = np.where(train_mask)[0]
test_mask = np.where(test_mask)[0]

train_graph = graph.subgraph(train_mask)
test_graph = graph.subgraph(test_mask)

# Step 7.1: Add self-loops to the graph
graph = dgl.add_self_loop(graph)

# Step 8: Define and train the GNN model
class GNN(nn.Module):
    def __init__(self, in_feats, hidden_size, num_classes):
        super(GNN, self).__init__()
        self.conv1 = dgl.nn.GraphConv(in_feats, hidden_size)
        self.conv2 = dgl.nn.GraphConv(hidden_size, num_classes)

    def forward(self, g, features):
        x = torch.relu(self.conv1(g, features))
        x = self.conv2(g, x)
        return x

in_feats = sensor_data.shape[1]  # Number of input features
hidden_size = 64
num_classes = 2  # Normal and anomaly
lr = 0.01
num_epochs = 100



NameError: name 'train_mask' is not defined

In [5]:
in_feats = sensor_data.shape[1]  # Number of input features
hidden_size = 64
num_classes = 2  # Normal and anomaly
lr = 0.01
num_epochs = 100
print(in_feats)

51


In [ ]:
model = GNN(in_feats, hidden_size, num_classes)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

def train(model, graph, features, labels, optimizer, criterion):
    model.train()
    optimizer.zero_grad()
    logits = model(graph, features)
    loss = criterion(logits, labels)
    loss.backward()
    optimizer.step()
    return loss.item()

def evaluate(model, graph, features, labels):
    model.eval()
    with torch.no_grad():
        logits = model(graph, features)
        _, predicted = torch.max(logits, dim=1)
        accuracy = (predicted == labels).float().mean().item()
        report = classification_report(labels, predicted, target_names=['NORMAL', 'ANOMALY'])
        cm = confusion_matrix(labels, predicted)
    return accuracy, report, cm

train_features = train_graph.ndata['features']
train_labels = train_graph.ndata['label']
test_features = test_graph.ndata['features']
test_labels = test_graph.ndata['label']

train_losses = []
test_losses = []
for epoch in range(num_epochs):
    train_loss = train(model, train_graph, train_features, train_labels, optimizer, criterion)
    test_accuracy, test_report, test_cm = evaluate(model, test_graph, test_features, test_labels)
    train_losses.append(train_loss)
    test_losses.append(test_accuracy)
    print(f'Epoch: {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

# Step 9: Visualize confusion matrix
labels = ['NORMAL', 'ANOMALY']

plt.figure(figsize=(10, 6))
plt.subplot(1, 2, 1)
sns.heatmap(train_cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.title('Train Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')

plt.subplot(1, 2, 2)
sns.heatmap(test_cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.title('Test Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')

plt.tight_layout()
plt.show()

In [ ]:
import warnings
from sklearn.metrics import confusion_matrix, classification_report

with warnings.catch_warnings():
    warnings.simplefilter("ignore")  # Ignore the UndefinedMetricWarning
    train_cm = confusion_matrix(y_train_encoded, train_predictions, labels=[0, 1])
    test_cm = confusion_matrix(y_test_encoded, test_predictions, labels=[0, 1])

train_report = classification_report(y_train_encoded, train_predictions, labels=[0, 1])
test_report = classification_report(y_test_encoded, test_predictions, labels=[0, 1])

print("Train Classification Report:\n", train_report)
print("Test Classification Report:\n", test_report)